In [0]:
import pandas as pd
import requests
import json
import networkx as nx
from pandas.io.json import json_normalize
import numpy as np

# @@@@@ OEC's API   --> https://atlas.media.mit.edu/api/
# @@@@@ GitHub repo --> https://github.com/mrmorais/USACHN-trade-war

In [0]:
## Helper function to fetch JSON data from URL
def df_from_json(url):
    # https://www.geeksforgeeks.org/pandas-parsing-json-dataset/
    with requests.get(url) as data:
        json_data = data.json()
    return json_normalize(json_data['data']) 

In [0]:
## Fetch a clean dataset
def df_from_json_clean(url):
    df = df_from_json(url)
    sitc_attr = df_from_json('https://atlas.media.mit.edu/attr/sitc/')
    df = df.merge(sitc_attr, left_on='sitc_id', right_on='id')
    
    ## Remove unused data
    usefull_columns = ['dest_id', 'export_val', 'import_val', 'origin_id', 'sitc_id', 'year', 'color', 'icon', 'name']
    df = df.drop(set(df.columns)-set(usefull_columns), 1)
    
    return df

In [0]:
## Takes averaged, normalized and filtered by only export_val and import_val, dataframe
def to_normalized_df(df):
    df = df.groupby(['name']).mean() # takes average by product classification
    df = df.drop('year', 1)
    
    # Normalizing data
    df = ((df-df.min())/(df.max()-df.min()))*10e5
    
    df['export_val'] = df['export_val'].fillna(0).astype(np.int)
    df['import_val'] = df['import_val'].fillna(0).astype(np.int)
    
    return df

In [0]:
## Adds product nodes and edges product-country
def add_relations_to_graph(graph, df, label_src, label_tgt):
    for product, item in df.iterrows():
        graph.add_node(product, chn_export=int(item.export_val))
        graph.add_edge(label_src, product, export_val=int(item.export_val))
        graph.add_edge(product, label_tgt, export_val=int(item.export_val))

## Brazil and USA to China

In [0]:
## USA TO CHN
usa_chn = df_from_json_clean('https://atlas.media.mit.edu/sitc/export/2012.2017/usa/chn/show/')
usa_chn = to_normalized_df(usa_chn)
## BRA TO CHN
bra_chn = df_from_json_clean('https://atlas.media.mit.edu/sitc/export/2012.2017/bra/chn/show/')
bra_chn = to_normalized_df(bra_chn)

In [0]:
graph_usabra_chn = nx.DiGraph()

In [0]:
# USA to CHN
add_relations_to_graph(graph_usabra_chn, usa_chn, "USA", "CHN")
# BRA to CHN
add_relations_to_graph(graph_usabra_chn, bra_chn, "BRA", "CHN")

In [63]:
graph_usabra_chn['USA']

AtlasView({'Abrasive Powder': {'export_val': 1838}, 'Acrylic Polymers': {'export_val': 9051}, 'Acyclic Alcohols': {'export_val': 20689}, 'Acyclic Hydrocarbons': {'export_val': 5002}, 'Air Conditioners': {'export_val': 2666}, 'Air Pumps and Compressors': {'export_val': 13894}, 'Aircraft Parts and Accessories': {'export_val': 3330}, 'Aircraft Tires': {'export_val': 360}, 'Aldehyde, Ketone and Quinone-Function Compounds': {'export_val': 4402}, 'Aluminium Structures': {'export_val': 480}, 'Aluminum': {'export_val': 552}, 'Aluminum Ore': {'export_val': 2744}, 'Amide Function Compounds': {'export_val': 591}, 'Amine-Function Compounds': {'export_val': 8498}, 'Aminoplasts': {'export_val': 2597}, 'Analog Instruments for Physical Analysis': {'export_val': 47099}, 'Analog Navigation Devices': {'export_val': 10343}, 'Animal meat, misc': {'export_val': 27}, 'Anthracite': {'export_val': 2}, 'Anti-Corrosive Chemicals': {'export_val': 12770}, 'Antibiotics': {'export_val': 2680}, 'Apples': {'export_val

In [0]:
nx.write_gexf(graph_usabra_chn, "usabra_chn.gexf")

## Brazil and China to USA

In [0]:
## CHN TO USA
chn_usa = df_from_json_clean('https://atlas.media.mit.edu/sitc/export/2012.2017/chn/usa/show/')
chn_usa = to_normalized_df(chn_usa)
## BRA TO USA
bra_usa = df_from_json_clean('https://atlas.media.mit.edu/sitc/export/2012.2017/bra/usa/show/')
bra_usa = to_normalized_df(bra_usa)

In [0]:
graph_chnbra_usa = nx.DiGraph()

In [0]:
# CHN to EUA
add_relations_to_graph(graph_chnbra_usa, chn_usa, "CHN", "USA")
# BRA to EUA    
add_relations_to_graph(graph_chnbra_usa, bra_usa, "BRA", "USA")

In [68]:
graph_chnbra_usa['CHN']

AtlasView({'Abrasive Powder': {'export_val': 936}, 'Acrylic Polymers': {'export_val': 1939}, 'Acyclic Alcohols': {'export_val': 2331}, 'Acyclic Hydrocarbons': {'export_val': 2022}, 'Air Conditioners': {'export_val': 56704}, 'Air Pumps and Compressors': {'export_val': 19928}, 'Aircraft': {'export_val': 968}, 'Aircraft Parts and Accessories': {'export_val': 15703}, 'Aircraft Tires': {'export_val': 3}, 'Aldehyde, Ketone and Quinone-Function Compounds': {'export_val': 9805}, 'Aluminium Structures': {'export_val': 7261}, 'Aluminum': {'export_val': 99}, 'Aluminum Ore': {'export_val': 565}, 'Amide Function Compounds': {'export_val': 6822}, 'Amine-Function Compounds': {'export_val': 6133}, 'Aminoplasts': {'export_val': 4605}, 'Analog Instruments for Physical Analysis': {'export_val': 6210}, 'Analog Navigation Devices': {'export_val': 3617}, 'Animal meat, misc': {'export_val': 129}, 'Anthracite': {'export_val': 5}, 'Anti-Corrosive Chemicals': {'export_val': 69}, 'Antibiotics': {'export_val': 42

In [0]:
nx.write_gexf(graph_chnbra_usa, "chnbra_usa.gexf")